# Video Summarization from LLM

In [38]:
import warnings
warnings.filterwarnings("ignore")

In [39]:
# for setting up language model we are using Together.ai because it is open-source and has specialized multimodal models for chat, images, code, and more. 

import os
from dotenv import load_dotenv #loads all env variable
from langchain_together import ChatTogether

load_dotenv(override= True)

llm = ChatTogether(api_key= "Your_API_KEY", temperature= 0.0, model = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")
#temperature controls randomness in response. 0 means deterministic
 


In [40]:
from langchain_community.document_loaders import YoutubeLoader #https://python.langchain.com/docs/integrations/document_loaders/youtube_transcript/
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=QsYGlZkevEg", add_video_info=False
)
data = loader.load()

In [41]:
data[0].page_content

'LADIES AND GENTLEMEN, PEDRO PASCAL! [ CHEERS AND APPLAUSE ] >> THANK YOU, THANK YOU. THANK YOU VERY MUCH. I\'M SO EXCITED TO BE HERE. THANK YOU. I SPENT THE LAST YEAR SHOOTING A SHOW CALLED "THE LAST OF US" ON HBO. FOR SOME HBO SHOES, YOU GET TO SHOOT IN A FIVE STAR ITALIAN RESORT SURROUNDED BY BEAUTIFUL PEOPLE, BUT I SAID, NO, THAT\'S TOO EASY. I WANT TO SHOOT IN A FREEZING CANADIAN FOREST WHILE BEING CHASED AROUND BY A GUY WHOSE HEAD LOOKS LIKE A GENITAL WART. IT IS AN HONOR BEING A PART OF THESE HUGE FRANCHISEs LIKE "GAME OF THRONES" AND "STAR WARS," BUT I\'M STILL GETTING USED TO PEOPLE RECOGNIZING ME. THE OTHER DAY, A GUY STOPPED ME ON THE STREET AND SAYS, MY SON LOVES "THE MANDALORIAN" AND THE NEXT THING I KNOW, I\'M FACE TIMING WITH A 6-YEAR-OLD WHO HAS NO IDEA WHO I AM BECAUSE MY CHARACTER WEARS A MASK THE ENTIRE SHOW. THE GUY IS LIKE, DO THE MANDO VOICE, BUT IT\'S LIKE A BEDROOM VOICE. WITHOUT THE MASK, IT JUST SOUNDS PORNY. PEOPLE WALKING BY ON THE STREET SEE ME WHISPERING T

In [42]:
messages = [("system",
             ''' provide summary of the youtube link provided. If data is given then donot forget to mention it ''' 
             ,),
            ("human", data[0].page_content),]

ai_msg=llm.invoke(messages)
ai_msg

AIMessage(content='The YouTube link appears to be a clip from an award show where Pedro Pascal is being honored. Here\'s a summary of the clip:\n\nPedro Pascal thanks the audience and expresses his excitement to be there. He talks about his recent work on the HBO show "The Last of Us," which he shot in a freezing Canadian forest. He also mentions being part of other huge franchises like "Game of Thrones" and "Star Wars."\n\nPascal shares a funny anecdote about being recognized by a fan who stopped him on the street and FaceTimed with his 6-year-old son, who was a fan of "The Mandalorian." He also talks about his Chilean heritage and his 34 first cousins who are still in Chile.\n\nHe mentions that his cousins weren\'t always proud of him, as he started his career playing small parts in crime shows, including two different characters on "Law and Order." However, he\'s grateful to be where he is today and thanks his parents for their bravery in fleeing Chile and bringing him to the US.\n\

In [43]:
# Invoking prompt template 
# consists of: template string for main instructions 
#              input variables for direct pass of youtube transcript as i/p

In [44]:
from langchain.prompts import PromptTemplate
from langchain import LLMChain

summariser_template = PromptTemplate(

    input_variables= ["video_transcript"],
    template= """ provide summary of the youtube link provided. 
    If data is given then donot forget to mention it
    Video transcript : {video_transcript}
     """
    
)

In [45]:
# using llmchain for summarization

chain = LLMChain(llm=llm,prompt = summariser_template) #to invoke chain with video transcript

summary = chain.invoke({
    "video_transcript": data[0].page_content
})

In [46]:
summary['text']

'The YouTube link provided is a video of Pedro Pascal, an actor known for his roles in "The Last of Us" and "The Mandalorian." \n\nSummary of the video:\n\nPedro Pascal expresses his gratitude for being at the event and talks about his recent projects, including "The Last of Us" on HBO. He shares a humorous anecdote about being recognized by a fan who stopped him on the street and FaceTimed with his 6-year-old son, who is a fan of "The Mandalorian."\n\nPascal also talks about his background, being born in Chile and moving to the US with his parents at a young age. He mentions that his 34 first cousins in Chile are very proud of him and often text him, asking for favors.\n\nHe also shares a lighthearted story about his early career, playing small parts in crime shows, including two different characters on "Law and Order." He jokes about the concept of "range" in acting.\n\nPascal expresses his gratitude to his parents for their bravery in fleeing Chile and bringing him and his sister to

In [47]:
from IPython.display import Markdown, display

display (Markdown(summary['text']))

The YouTube link provided is a video of Pedro Pascal, an actor known for his roles in "The Last of Us" and "The Mandalorian." 

Summary of the video:

Pedro Pascal expresses his gratitude for being at the event and talks about his recent projects, including "The Last of Us" on HBO. He shares a humorous anecdote about being recognized by a fan who stopped him on the street and FaceTimed with his 6-year-old son, who is a fan of "The Mandalorian."

Pascal also talks about his background, being born in Chile and moving to the US with his parents at a young age. He mentions that his 34 first cousins in Chile are very proud of him and often text him, asking for favors.

He also shares a lighthearted story about his early career, playing small parts in crime shows, including two different characters on "Law and Order." He jokes about the concept of "range" in acting.

Pascal expresses his gratitude to his parents for their bravery in fleeing Chile and bringing him and his sister to the US. He ends by speaking in Spanish to his family in Chile, saying "I love you, I miss you, and stop giving out my phone number."

Data: 
- Pedro Pascal was born in Chile.
- He has 34 first cousins in Chile.
- He played two different characters on "Law and Order" within a year of each other.
- He has been a part of huge franchises like "Game of Thrones" and "Star Wars."
- He has been a part of shows like "The Last of Us" and "The Mandalorian."